### **Импорт необходимых библиотек**

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

import requests
from bs4 import BeautifulSoup
url = 'https://ru.investing.com/earnings-calendar/'
base_url = "https://ru.investing.com/"
response = requests.get(url)
tree = BeautifulSoup(response.content, 'html.parser')

### **Парсинг данных**

In [ ]:
companies = tree.find_all("td",{'class' : 'left noWrap earnCalCompany'})
len(companies)

748

In [ ]:
companies_dict = dict()
for i in range(len(companies)):
  full_company_name = companies[i].get('title')
  company_name = companies[i].span.text
  link_ending = companies[i].a.get('href')[1:]
  link = base_url + link_ending
  companies_dict[full_company_name] = link

In [ ]:
div_companies = dict()
for full_company_name, link in companies_dict.items():
  div_link = link[:-8] + 'dividends'
  div_companies[full_company_name] = div_link

In [ ]:
balance_sheets_to_parse = dict()
income_stataments_to_parse = dict()
for i in range(len(companies)):
  company_id = companies[i].get('_r_pid')
  balance_to_parse = 'https://ru.investing.com/instruments/Financials/changereporttypeajax?action=change_report_type&pair_ID='+company_id + '&report_type=' + 'BAL' + '&period_type=Annual'
  income_to_parse = 'https://ru.investing.com/instruments/Financials/changereporttypeajax?action=change_report_type&pair_ID=' + company_id + '&report_type=' + 'INC' + '&period_type=Annual'
  company_name = companies[i].get('title')
  balance_sheets_to_parse[company_name] = balance_to_parse
  income_stataments_to_parse[company_name] = income_to_parse

len(balance_sheets_to_parse)

719

In [ ]:
import itertools
dict(itertools.islice(balance_sheets_to_parse.items(), 3))

{'ОАО НОВАТЭК': 'https://ru.investing.com/instruments/Financials/changereporttypeajax?action=change_report_type&pair_ID=13697&report_type=BAL&period_type=Annual',
 'ГМК Норильский Никель ПАО': 'https://ru.investing.com/instruments/Financials/changereporttypeajax?action=change_report_type&pair_ID=13683&report_type=BAL&period_type=Annual',
 'ОАО Северсталь': 'https://ru.investing.com/instruments/Financials/changereporttypeajax?action=change_report_type&pair_ID=13713&report_type=BAL&period_type=Annual'}

In [ ]:
dict(itertools.islice(income_stataments_to_parse.items(), 3))

{'ОАО НОВАТЭК': 'https://ru.investing.com/instruments/Financials/changereporttypeajax?action=change_report_type&pair_ID=13697&report_type=INC&period_type=Annual',
 'ГМК Норильский Никель ПАО': 'https://ru.investing.com/instruments/Financials/changereporttypeajax?action=change_report_type&pair_ID=13683&report_type=INC&period_type=Annual',
 'ОАО Северсталь': 'https://ru.investing.com/instruments/Financials/changereporttypeajax?action=change_report_type&pair_ID=13713&report_type=INC&period_type=Annual'}

In [ ]:
# dividends_dict = dict()
# for company, link in companies_dict.items():
#   company_resp = requests.get(link)
#   company_tree = BeautifulSoup(company_resp.content, 'html.parser')
#   div_url = company_tree.find_all('ul', {'class' : 'arial_12'})[0].find_all('a')[-1].get('href')
#   full_div_url = base_url[:-1]+div_url
#   dividends_dict[company] = full_div_url

In [ ]:
# div_companies
# balance_sheets_to_parse
# income_stataments_to_parse

In [ ]:
df = pd.DataFrame()
cnt_errors = 0
for company, balance_to_parse in balance_sheets_to_parse.items():
  balance_resp = requests.get(balance_to_parse)
  balance_tree = BeautifulSoup(balance_resp.content, 'html.parser')
  # нашли табличку
  # balance_table = balance_tree.find('table')
  try:
    balance_table = balance_tree.find_all('table')[0]

    # распарсили её
    balance_df = pd.read_html(str(balance_table))[0]

    balance_df.columns = [balance_tree.find_all('th')[0].span.text, balance_tree.find_all('th')[1].span.text, balance_tree.find_all('th')[2].span.text, balance_tree.find_all('th')[3].span.text, balance_tree.find_all('th')[4].span.text]
    new_balance_df = balance_df.drop([1,12, 23,31,39], axis=0, inplace=False)
    new_balance_df.replace('-', np.NaN, inplace=True)
    # года в численные значения
    new_balance_df[balance_tree.find_all('th')[1].span.text] = new_balance_df[balance_tree.find_all('th')[1].span.text].apply(pd.to_numeric, errors='coerce')
    new_balance_df[balance_tree.find_all('th')[2].span.text] = new_balance_df[balance_tree.find_all('th')[2].span.text].apply(pd.to_numeric, errors='coerce')
    new_balance_df[balance_tree.find_all('th')[3].span.text] = new_balance_df[balance_tree.find_all('th')[3].span.text].apply(pd.to_numeric, errors='coerce')
    new_balance_df[balance_tree.find_all('th')[4].span.text] = new_balance_df[balance_tree.find_all('th')[4].span.text].apply(pd.to_numeric, errors='coerce')
    # заменить NaN на 0 в строке
    new_balance_df.iloc[:,1:] = new_balance_df.iloc[:, 1:].apply(lambda row: row.fillna(0), axis=1)
    new_balance_df = new_balance_df.rename(columns = {'Период до:': company})
  except:
    cnt_errors +=1
  df = pd.concat([df,new_balance_df],axis=1)
print(f'Я не смог распарсить {cnt_errors} компаний')

я не смог распарсить 21 компаний


In [ ]:
df

,Период до:,2018,2017,2016,2015,Период до:,2018,2017,2016,2015,...,Период до:,2018,2017,2016,2015,Yuma Energy Inc,2018,2017,2016,2015
0,Итого оборотные активы,717,702,1163,1289,Итого оборотные активы,717,702,1163,1289,...,Итого оборотные активы,717,702,1163,1289,Итого оборотные активы,717.0,702.0,1163.0,1289.0
2,Денежные средства и краткосрочные финансовые в...,163,014,363,406,Денежные средства и краткосрочные финансовые в...,163,014,363,406,...,Денежные средства и краткосрочные финансовые в...,163,014,363,406,Денежные средства и краткосрочные финансовые в...,163.0,14.0,363.0,406.0
3,Денежные средства,163,014,0,0,Денежные средства,163,014,0,0,...,Денежные средства,163,014,0,0,Денежные средства,163.0,14.0,0.0,0.0
4,Денежные средства и их эквиваленты,163,014,363,406,Денежные средства и их эквиваленты,163,014,363,406,...,Денежные средства и их эквиваленты,163,014,363,406,Денежные средства и их эквиваленты,163.0,14.0,363.0,406.0
5,Краткосрочные финансовые вложения,0,0,0,0,Краткосрочные финансовые вложения,0,0,0,0,...,Краткосрочные финансовые вложения,0,0,0,0,Краткосрочные финансовые вложения,0.0,0.0,0.0,0.0
6,"Общая дебиторская задолженность, нетто",338,555,665,678,"Общая дебиторская задолженность, нетто",338,555,665,678,...,"Общая дебиторская задолженность, нетто",338,555,665,678,"Общая дебиторская задолженность, нетто",338.0,555.0,665.0,678.0
7,"Дебиторская задолженность - торговля, нетто",318,45,483,299,"Дебиторская задолженность - торговля, нетто",318,45,483,299,...,"Дебиторская задолженность - торговля, нетто",318,45,483,299,"Дебиторская задолженность - торговля, нетто",318.0,45.0,483.0,299.0
8,Итого Товарно-материальные Запасы,0,0,0,0,Итого Товарно-материальные Запасы,0,0,0,0,...,Итого Товарно-материальные Запасы,0,0,0,0,Итого Товарно-материальные Запасы,0.0,0.0,0.0,0.0
9,Расходы будущих периодов,115,098,106,033,Расходы будущих периодов,115,098,106,033,...,Расходы будущих периодов,115,098,106,033,Расходы будущих периодов,115.0,98.0,106.0,33.0
10,"Прочие оборотные активы, всего",101,035,028,171,"Прочие оборотные активы, всего",101,035,028,171,...,"Прочие оборотные активы, всего",101,035,028,171,"Прочие оборотные активы, всего",101.0,35.0,28.0,171.0


### **Шаг 2. EDA**

In [ ]:
#проверим пропуски
skip = df.isnull().sum()
skip

Период до:         0
2018               0
2017               0
2016               0
2015               0
                  ..
Yuma Energy Inc    0
2018               0
2017               0
2016               0
2015               0
Length: 3595, dtype: int64